# Classification de documents

## Imports

In [6]:
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
#import seaborn as sn
from pprint import pprint
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
%pip install scikit-plot
from scikitplot.metrics import plot_confusion_matrix
import pandas as pd
import re
import operator
import nltk
nltk.download('stopwords')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\janvi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Charger le dataset 20 newsgroups
Pour plus d'information : https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

In [7]:
news = fetch_20newsgroups(subset='all')

In [8]:
print("Number of articles: " + str(len(news.data)))

Number of articles: 18846


In [9]:
print("Number of categories: " + str(len(news.target_names)))

Number of categories: 20


In [10]:
labels = news.target_names
print(labels)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [11]:
# Exemples d'articles et de labels
for i, article in enumerate(news.data[:10]):
    print(f'===== {labels[news.target[i]]} =====')
    print(article.replace('\n', ' '), '\n')

===== rec.sport.hockey =====
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu> Subject: Pens fans reactions Organization: Post Office, Carnegie Mellon, Pittsburgh, PA Lines: 12 NNTP-Posting-Host: po4.andrew.cmu.edu    I am sure some bashers of Pens fans are pretty confused about the lack of any kind of posts about the recent Pens massacre of the Devils. Actually, I am  bit puzzled too and a bit relieved. However, I am going to put an end to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they are killing those Devils worse than I thought. Jagr just showed you why he is much better than his regular season stats. He is also a lot fo fun to watch in the playoffs. Bowman should let JAgr have a lot of fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final regular season game.          PENS RULE!!!   

===== comp.sys.ibm.pc.hardware =====
From: mblawson@midway.ecn.uok

## Création d'un modèle de machine learning avec Scikit-Learn
Pour plus d'information :
- Pipeline : https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
- TfidfVectorizer : https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
- MultinomialNB : https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

Un article de blog qui explique le TFIDF:
- https://medium.com/analytics-vidhya/tf-idf-term-frequency-technique-easiest-explanation-for-text-classification-in-nlp-with-code-8ca3912e58c3

Un article de blog qui explique les naive bayes:
- https://towardsdatascience.com/naive-bayes-classifier-explained-54593abe6e18

### Séparer le dataset en features et target (X, y) et en train et test
Plus d'information : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [12]:
# Nettoyage des textes
texts = [re.sub('[^a-z]+', ' ', t.lower()).strip() for t in news.data]

In [13]:
# Mapping des targets
targets = np.array([labels[t] for t in news.target])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(texts, targets, test_size=0.2, random_state=11)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))

Training set size: 15076
Test set size: 3770


### Entrainer un modèle de machine learning sur les données d'entrainement

In [15]:
# Définition du type de modèle
classifier = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stopwords.words('english'), min_df=50, max_df=0.5)),
    ('classifier', MultinomialNB()),
])

In [16]:
# Entrainement du modèle
classifier.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=0.5, min_df=50,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('classifier', MultinomialNB())])

### Qu'est ce qu'il s'est passé ?

#### Le TFIDF calcule le score IDF de chaque mot du corpus


In [ ]:
feature_names = classifier.named_steps['vectorizer'].get_feature_names_out()
idf_scores = classifier.named_steps['vectorizer'].idf_

In [ ]:
# Taille du vocabulaire
len(feature_names)

In [ ]:
# Score IDF de chaque terme du vocabulaire
for i in range(0, 10):
    print(feature_names[i], ':', round(idf_scores[i], 2))

In [ ]:
# Les 10 mots avec le score IDF le plus haut
for word, score in sorted(zip(feature_names, idf_scores), key=operator.itemgetter(1), reverse=True)[:20]:
    print(word, round(score, 2))

#### Le TF-IDF transforme chaque document en vecteur de la taille du vocabulaire et donc le score est le TFIDF (fréquence du terme dans le document * idf)

In [ ]:
tmp = classifier.named_steps['vectorizer'].transform(X_train[:10])
pd.DataFrame(tmp.toarray(), columns=classifier.named_steps['vectorizer'].get_feature_names_out())

#### Le modèle naïf bayésien apprend la corrélation entre chaque mot et chaque catégorie

In [ ]:
pd.DataFrame(classifier.named_steps['classifier'].feature_log_prob_, index=labels, columns=feature_names).T

#### On peut ainsi découvrir les termes les plus contributifs pour un label donné

In [ ]:
pd.DataFrame(classifier.named_steps['classifier'].feature_log_prob_, index=labels, columns=feature_names).T.sort_values(by='comp.graphics', ascending=False).head(20)

### Prédire les targets des données de test à l'aide du modèle entrainé

In [ ]:
y_pred = classifier.predict(X_test)

Aperçu des targets prédites

In [ ]:
y_pred[:20]

Aperçu des targets réelles

In [ ]:
y_test[:20]

### Evaluer le modèle

#### Générer un rapport de classification
Pour plus d'information sur la précision, le recall et le f1-score : https://fr.wikipedia.org/wiki/Pr%C3%A9cision_et_rappel

In [ ]:
print(classification_report(y_test, y_pred))

### Générer une matrice de confusion

In [ ]:
plot_confusion_matrix(y_test, y_pred, figsize=(10, 10), labels=labels, x_tick_rotation=90)